In [1]:
import pandas as pd
import csv
from sqlalchemy import create_engine

In [2]:
tsv_data = pd.read_csv('county_market_tracker.tsv000', sep='\t')
tsv_data.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2016-10-01,2016-10-31,30,county,5,1356,f,"Bay County, MI",NaN,Michigan,...,0.000000,0.045455,NaN,NaN,0.000000,0.000000,0.000000,"Bay City, MI",13020.0,2022-03-13 14:34:13
1,2015-01-01,2015-01-31,30,county,5,2406,f,"Montgomery County, PA",NaN,Pennsylvania,...,-0.000673,0.199631,0.044288,0.005287,0.337696,0.155264,-0.003086,"Montgomery County, PA",33874.0,2022-03-13 14:34:13
2,2016-01-01,2016-01-31,30,county,5,2934,f,"Washington County, VT",NaN,Vermont,...,0.000000,0.067797,0.033898,NaN,0.000000,0.000000,NaN,"Barre, VT",12740.0,2022-03-13 14:34:13
3,2020-12-01,2020-12-31,30,county,5,2605,f,"Meigs County, TN",NaN,Tennessee,...,0.400000,NaN,NaN,NaN,0.000000,0.000000,0.000000,Tennessee nonmetropolitan area,NaN,2022-03-13 14:34:13
4,2017-10-01,2017-10-31,30,county,5,155,f,"Jefferson County, AL",NaN,Alabama,...,-0.021642,0.185209,0.001860,0.006144,0.163311,0.055688,-0.079451,"Birmingham, AL",13820.0,2022-03-13 14:34:13


In [3]:
california_df=tsv_data.loc[(tsv_data["state"]=="California")]
california_df=california_df.sort_values(["region","period_end"],ascending=True).fillna(0)
california_df[['Year','Month','Day']] = california_df['period_end'].str.split('-', expand=True).astype(int)
california_df.columns

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

In [4]:
cleaned_ca_df=california_df[['Year','Month', 'region', 'state', 'state_code', 'property_type','median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy','median_list_price', 'median_list_price_mom', 'median_list_price_yoy','median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf','median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold','homes_sold_mom', 'homes_sold_yoy', 'pending_sales','pending_sales_mom', 'pending_sales_yoy', 'new_listings','new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom','inventory_yoy', 'months_of_supply', 'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom', 'median_dom_mom','median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom','avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom','sold_above_list_yoy', 'price_drops', 'price_drops_mom', 'price_drops_yoy', 'off_market_in_two_weeks', 'off_market_in_two_weeks_mom', 'off_market_in_two_weeks_yoy','parent_metro_region', 'parent_metro_region_metro_code', 'last_updated']]
cleaned_ca_df.head()
cleaned_ca_df.to_csv("california_housing")

In [84]:
single_family_df=cleaned_ca_df.loc[(cleaned_ca_df['property_type']=='Single Family Residential')].reset_index(drop=True)
townhouse_df=cleaned_ca_df.loc[(cleaned_ca_df['property_type']=='Townhouse')].reset_index(drop=True)
all_residential_df=cleaned_ca_df.loc[(cleaned_ca_df['property_type']=='All Residential')].reset_index(drop=True)
condo_df=cleaned_ca_df.loc[(cleaned_ca_df['property_type']=='Multi-Family (2-4 Unit)')].reset_index(drop=True)
multi_family_df=cleaned_ca_df.loc[(cleaned_ca_df['property_type']=='Condo/Co-op')].reset_index(drop=True)
single_family_df

orange_df=single_family_df.loc[(single_family_df['region']=='Orange County, CA') & (single_family_df['Year']>=2020)]
orange_df


,Year,Month,region,state,state_code,property_type,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
2387,2020,1,"Orange County, CA",California,CA,Single Family Residential,850000.0,0.017964,0.082803,770000.0,...,0.025608,0.146321,0.033768,-0.079492,0.241501,0.079812,0.052311,"Anaheim, CA",11244.0,2022-03-13 14:34:13
2388,2020,2,"Orange County, CA",California,CA,Single Family Residential,870000.0,0.023529,0.108280,795000.0,...,0.109642,0.137839,-0.008482,-0.043429,0.293358,0.051857,0.066691,"Anaheim, CA",11244.0,2022-03-13 14:34:13
2389,2020,3,"Orange County, CA",California,CA,Single Family Residential,870000.0,0.000000,0.087500,779888.0,...,0.125515,0.153948,0.016109,-0.049546,0.224693,-0.068665,0.007526,"Anaheim, CA",11244.0,2022-03-13 14:34:13
2390,2020,4,"Orange County, CA",California,CA,Single Family Residential,859000.0,-0.012644,0.053988,769000.0,...,0.118182,0.136024,-0.017925,-0.088619,0.152763,-0.071931,-0.032770,"Anaheim, CA",11244.0,2022-03-13 14:34:13
2391,2020,5,"Orange County, CA",California,CA,Single Family Residential,825000.0,-0.039581,-0.006024,785000.0,...,0.038437,0.176653,0.040630,-0.070818,0.243299,0.090536,0.065507,"Anaheim, CA",11244.0,2022-03-13 14:34:13
2392,2020,6,"Orange County, CA",California,CA,Single Family Residential,859000.0,0.041212,0.036814,779000.0,...,-0.005060,0.186764,0.010111,-0.072288,0.247781,0.004482,0.084738,"Anaheim, CA",11244.0,2022-03-13 14:34:13
2393,2020,7,"Orange County, CA",California,CA,Single Family Residential,875000.0,0.018626,0.054217,799000.0,...,0.068791,0.181839,-0.004925,-0.093347,0.257639,0.009858,0.098982,"Anaheim, CA",11244.0,2022-03-13 14:34:13
2394,2020,8,"Orange County, CA",California,CA,Single Family Residential,925000.0,0.057143,0.154086,789900.0,...,0.122099,0.181707,-0.000132,-0.092977,0.243332,-0.014308,0.083475,"Anaheim, CA",11244.0,2022-03-13 14:34:13
2395,2020,9,"Orange County, CA",California,CA,Single Family Residential,900500.0,-0.026486,0.104908,795000.0,...,0.149069,0.175115,-0.006592,-0.068693,0.232356,-0.010975,0.064125,"Anaheim, CA",11244.0,2022-03-13 14:34:13
2396,2020,10,"Orange County, CA",California,CA,Single Family Residential,910000.0,0.010550,0.123457,798000.0,...,0.167675,0.178418,0.003302,-0.062871,0.233815,0.001459,0.060662,"Anaheim, CA",11244.0,2022-03-13 14:34:13
